In [6]:
!python -V

Python 3.11.13


In [4]:
import pandas as pd

In [2]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import sklearn
print(sklearn.__version__)

1.7.1


In [8]:
import mlflow

# Using absolute path to the correct DB
mlflow.set_tracking_uri("sqlite:////workspaces/mlops-zoomcamp/mlflow.db") #Locally
mlflow.set_experiment("nyc-taxi-experiment")

# Verifying explicitly the experiment exists
print(mlflow.get_experiment_by_name("nyc-taxi-experiment"))

2025/07/31 22:02:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/31 22:02:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-mlflow/mlruns/1', creation_time=1753954436563, experiment_id='1', last_update_time=1753954436563, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>


In [9]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_train = pd.read_parquet('/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-02.parquet')

In [15]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00


In [16]:
len(df_train), len(df_val)

(76518, 64572)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
print("Train columns:", df_train.columns.tolist())
print("Validation columns:", df_val.columns.tolist())

Train columns: ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge', 'PU_DO']
Validation columns: ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge', 'PU_DO']


In [13]:
# Convert string timestamps to datetime if they're not already
df_train['lpep_pickup_datetime'] = pd.to_datetime(df_train['lpep_pickup_datetime'])
df_train['lpep_dropoff_datetime'] = pd.to_datetime(df_train['lpep_dropoff_datetime'])
df_val['lpep_pickup_datetime'] = pd.to_datetime(df_val['lpep_pickup_datetime'])
df_val['lpep_dropoff_datetime'] = pd.to_datetime(df_val['lpep_dropoff_datetime'])

# Calculate duration in minutes
df_train['duration'] = (df_train['lpep_dropoff_datetime'] - df_train['lpep_pickup_datetime']).dt.total_seconds() / 60
df_val['duration'] = (df_val['lpep_dropoff_datetime'] - df_val['lpep_pickup_datetime']).dt.total_seconds() / 60

# Now we can access the duration column
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

import numpy as np

mse = ((y_val - y_pred) ** 2).mean()  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error
print(f"RMSE: {rmse:.2f}")

RMSE: 53.60


In [22]:
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print(f"RMSE: {rmse:.2f}")

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "manza")

    mlflow.log_param("train-data-path", "/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-02.parquet")

    alpha = 0.1

    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    mse = ((y_val - y_pred) ** 2).mean()  # Mean Squared Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    print(f"RMSE: {rmse:.2f}")
    
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

RMSE: 53.15


PART 2 : COURSE XGBOOST EXAMPLE

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/miniforge3/envs/new-env/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [31]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:56.96564                          
[1]	validation-rmse:60.96017                          
[2]	validation-rmse:62.82297                          
[3]	validation-rmse:63.09372                          
[4]	validation-rmse:63.20058                          
[5]	validation-rmse:63.19900                          
[6]	validation-rmse:63.20125                          
[7]	validation-rmse:63.19889                          
[8]	validation-rmse:63.21319                          
[9]	validation-rmse:63.21244                          
[10]	validation-rmse:63.21190                         
[11]	validation-rmse:63.21085                         
[12]	validation-rmse:63.20998                         
[13]	validation-rmse:63.20895                         
[14]	validation-rmse:63.20831                         
[15]	validation-rmse:63.21702                         
[16]	validation-rmse:63.21679                         
[17]	validation-rmse:63.24771                         
[18]	valid

In [32]:
mlflow.xgboost.autolog(disable=True) # Autolog automatiquement sur MLFLOW - cons : debug diff à isoler le prob, logs perso non dispo avec autolog, controle etc

In [20]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:53.00786
[1]	validation-rmse:52.95303
[2]	validation-rmse:52.99269
[3]	validation-rmse:53.11095
[4]	validation-rmse:53.26545
[5]	validation-rmse:53.46482
[6]	validation-rmse:53.63349
[7]	validation-rmse:53.82890
[8]	validation-rmse:54.04723
[9]	validation-rmse:54.24796
[10]	validation-rmse:54.48840
[11]	validation-rmse:54.68821
[12]	validation-rmse:54.90494
[13]	validation-rmse:55.11320
[14]	validation-rmse:55.31020
[15]	validation-rmse:55.51602
[16]	validation-rmse:55.72216
[17]	validation-rmse:55.86983
[18]	validation-rmse:56.05304
[19]	validation-rmse:56.16449
[20]	validation-rmse:56.32612
[21]	validation-rmse:56.44342
[22]	validation-rmse:56.54786
[23]	validation-rmse:56.65801
[24]	validation-rmse:56.79654
[25]	validation-rmse:56.92093
[26]	validation-rmse:56.97124
[27]	validation-rmse:57.03038
[28]	validation-rmse:57.09771
[29]	validation-rmse:57.11133
[30]	validation-rmse:57.18427
[31]	validation-rmse:57.23150
[32]	validation-rmse:57.28320
[33]	validation-rmse

2025/07/31 22:24:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/miniforge3/envs/new-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [22:24:17] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/07/31 22:24:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
import pandas as pd
import mlflow
import xgboost as xgb
import pickle
import os
import numpy as np

params = {    
    'learning_rate': 0.1478262522723516,
    'max_depth': 4,
    'min_child_weight': 13.971201876547578,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.14143510877266188,
    'reg_lambda': 0.23656439070441268,
    'seed': 42
}

with mlflow.start_run():
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val,y_pred))
    mlflow.log_metric("rmse", rmse)

# Sauvegarde du modèle XGBoost en binaire
    os.makedirs("models", exist_ok=True)
    model_path = "models/xgb_model.bin"
    booster.save_model(model_path)

    # Log avec MLflow
    mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")

[0]	validation-rmse:52.85412
[1]	validation-rmse:52.62032
[2]	validation-rmse:52.45100
[3]	validation-rmse:52.32615
[4]	validation-rmse:52.23601
[5]	validation-rmse:52.16531
[6]	validation-rmse:52.11236
[7]	validation-rmse:52.06777
[8]	validation-rmse:52.04211
[9]	validation-rmse:52.02105
[10]	validation-rmse:52.00349
[11]	validation-rmse:51.99251
[12]	validation-rmse:51.99707
[13]	validation-rmse:51.98859
[14]	validation-rmse:51.98114
[15]	validation-rmse:51.98108
[16]	validation-rmse:51.97805
[17]	validation-rmse:51.98183
[18]	validation-rmse:51.97667
[19]	validation-rmse:51.98445
[20]	validation-rmse:51.98418
[21]	validation-rmse:51.98663
[22]	validation-rmse:51.99053
[23]	validation-rmse:51.99053
[24]	validation-rmse:51.99693
[25]	validation-rmse:51.99764
[26]	validation-rmse:51.99896
[27]	validation-rmse:52.00547
[28]	validation-rmse:52.00756
[29]	validation-rmse:52.00600
[30]	validation-rmse:52.00987
[31]	validation-rmse:52.02012
[32]	validation-rmse:52.02788
[33]	validation-rmse

/tmp/ipykernel_101779/3284102414.py:34: UserWarning: [16:34:46] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  booster.save_model(model_path)
